# Notebook TITANIC

<img src="https://kwmp.ca/wp-content/uploads/2018/04/titanic-the-musical-1024x538.jpg">

## Description des variables :
 - PassengerId: and id given to each traveler on the boat
 - Pclass: the passenger class. It has three possible values: 1,2,3 (first, second and third class)
 - The Name of the passeger
 - The Sex
 - The Age
 - SibSp: number of siblings and spouses traveling with the passenger
 - Parch: number of parents and children traveling with the passenger
 - The ticket number
 - The ticket Fare
 - The cabin number
 - The embarkation. This describe three possible areas of the Titanic from which the people embark. Three possible values S,C,Q

> https://www.kaggle.com/c/titanic

## Utilisation Azure ML service :

<img src="https://raw.githubusercontent.com/MicrosoftDocs/azure-docs/master/articles/machine-learning/service/media/overview-what-is-azure-ml/aml.png">

> Documentation : https://azure.microsoft.com/en-us/services/machine-learning-service/
<br>

### 0. Paramétrage

In [6]:
import datetime
datetime.datetime.now()
print("Date : " + str(datetime.datetime.now()))

Date : 2019-06-17 13:44:57.179357

In [7]:
import sys
sys.version

Out[ 2 ]: '3.5.2 (default, Nov 12 2018, 13:43:14) \n[GCC 5.4.0 20160609]'

In [8]:
import azureml.core
print("Version Azure ML Service :", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core:StepRun._from_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core:StepRun._from_reused_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core:PipelineRun._from_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Version Azure ML Service : 1.0.43

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas.io.sql as pd_sql

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

import logging
import os
import random

## 1. Chargement des données

In [11]:
url="https://raw.githubusercontent.com/retkowsky/titanic/master/train.csv"
train=pd.read_csv(url, index_col=None, na_values=['NA'])

# Suppression valeurs manquantes
train = train.dropna()

In [12]:
# Nombre de col et de lignes
print(train.shape)

(183, 12)

In [13]:
train.columns

Out[ 7 ]: 
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
 dtype='object')

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 1 to 889
Data columns (total 12 columns):
PassengerId 183 non-null int64
Survived 183 non-null int64
Pclass 183 non-null int64
Name 183 non-null object
Sex 183 non-null object
Age 183 non-null float64
SibSp 183 non-null int64
Parch 183 non-null int64
Ticket 183 non-null object
Fare 183 non-null float64
Cabin 183 non-null object
Embarked 183 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 18.6+ KB

In [15]:
display(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


In [16]:
# Suppression de colonnes
train = train.drop(['Cabin','Parch'],axis=1)

## 2. Analyses exploratoires

In [18]:
train.describe()

Out[ 10 ]: 
 PassengerId Survived Pclass Age SibSp Parch Fare
count 183.00 183.00 183.00 183.00 183.00 183.00 183.00
mean 455.37 0.67 1.19 35.67 0.46 0.48 78.68
std 247.05 0.47 0.52 15.64 0.64 0.75 76.35
min 2.00 0.00 1.00 0.92 0.00 0.00 0.00
25% 263.50 0.00 1.00 24.00 0.00 0.00 29.70
50% 457.00 1.00 1.00 36.00 0.00 0.00 57.00
75% 676.00 1.00 1.00 47.50 1.00 1.00 90.00
max 890.00 1.00 3.00 80.00 3.00 4.00 512.33

### Distribution variable Cible

In [20]:
display(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


In [21]:
display(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


## Tables Pivot

In [23]:
# Répartition par Sexe
train.groupby('Sex')[['Survived']].mean()

Out[ 13 ]: 
 Survived
Sex 
female 0.93
male 0.43

In [24]:
display(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


In [25]:
Age = pd.cut(train['Age'], [0, 18, 80])
train.pivot_table('Survived', ['Sex', Age], 'Pclass')

Out[ 15 ]: 
Pclass 1 2 3
Sex Age 
female (0, 18] 0.91 1.00 0.50
 (18, 80] 0.97 0.88 0.67
male (0, 18] 0.80 1.00 1.00
 (18, 80] 0.39 0.33 0.25

In [26]:
train.pivot_table('Survived', ['SibSp', Age], 'Pclass')

Out[ 16 ]: 
Pclass 1 2 3
SibSp Age 
0 (0, 18] 1.00 nan 0.50
 (18, 80] 0.60 0.73 0.50
1 (0, 18] 0.75 1.00 1.00
 (18, 80] 0.73 nan 0.00
2 (0, 18] 1.00 1.00 nan
 (18, 80] 0.67 nan nan
3 (18, 80] 0.67 nan nan

In [27]:
display(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


### Distribution variable pclass

In [29]:
train.pivot_table('Survived', index='Pclass')

Out[ 18 ]: 
 Survived
Pclass 
1 0.67
2 0.80
3 0.50

In [30]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Out[ 19 ]: 
 Pclass Survived
1 2 0.80
0 1 0.67
2 3 0.50

In [31]:
display(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


### Distribution variable sexe

In [33]:
train.pivot_table('Survived', index='Sex')

Out[ 21 ]: 
 Survived
Sex 
female 0.93
male 0.43

In [34]:
display(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


### Distribution variable Sexe en %

In [36]:
display(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0,D56,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


### Analyse de la cible

In [38]:
print("\n\nNumber of passengers in each classes: \n",train['Pclass'].value_counts())
train['Pclass'].value_counts().sort_index().plot(kind='bar')
train.groupby('Pclass').mean()

Number of passengers in each classes: 
 1 158
2 15
3 10
Name: Pclass, dtype: int64
 Out[ 24 ]: 
 PassengerId Survived Age SibSp Parch Fare
Pclass 
1 467.22 0.67 37.59 0.49 0.47 88.68
2 343.27 0.80 25.27 0.40 0.40 18.44
3 436.20 0.50 21.00 0.20 0.70 11.03

In [39]:
train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Out[ 25 ]: 
 Sex Survived
0 female 0.93
1 male 0.43

In [40]:
genderclass = train.groupby(['Pclass','Sex']).mean()
genderclass

Out[ 26 ]: 
 PassengerId Survived Age SibSp Parch Fare
Pclass Sex 
1 female 472.78 0.96 34.09 0.58 0.55 103.13
 male 462.32 0.42 40.67 0.40 0.39 75.96
2 female 440.67 0.89 29.61 0.22 0.11 14.87
 male 197.17 0.67 18.75 0.67 0.83 23.81
3 female 337.60 0.60 17.20 0.40 1.20 13.36
 male 534.80 0.40 24.80 0.00 0.20 8.70

In [41]:
display(genderclass)

PassengerId,Survived,Age,SibSp,Parch,Fare
472.7837837837838,0.9594594594594594,34.0945945945946,0.581081081081081,0.5540540540540541,103.12820945945943
462.32142857142856,0.4166666666666667,40.67166666666667,0.40476190476190477,0.39285714285714285,75.9578880952381
440.6666666666667,0.8888888888888888,29.61111111111111,0.2222222222222222,0.1111111111111111,14.865744444444443
197.16666666666666,0.6666666666666666,18.75,0.6666666666666666,0.8333333333333334,23.8125
337.6,0.6,17.2,0.4,1.2,13.36
534.8,0.4,24.8,0.0,0.2,8.695


## 3. Modélisation AutoML avec Azure ML service

<img src="https://blog.cellenza.com/wp-content/uploads/2018/11/automl.png">

In [44]:
col_target=['Survived']
col_train=['Age','Pclass','Sex','Fare','SibSp']
X=train[col_train]
y=train[col_target]

In [45]:
X.describe()

Out[ 29 ]: 
 Age Pclass Fare SibSp
count 183.00 183.00 183.00 183.00
mean 35.67 1.19 78.68 0.46
std 15.64 0.52 76.35 0.64
min 0.92 1.00 0.00 0.00
25% 24.00 1.00 29.70 0.00
50% 36.00 1.00 57.00 0.00
75% 47.50 1.00 90.00 1.00
max 80.00 3.00 512.33 3.00

In [46]:
dic = {'male':0,'female':1}
X['Sex']=X['Sex'].apply(lambda x:dic[x])

In [47]:
display(X)

Age,Pclass,Sex,Fare,SibSp
38.0,1,1,71.2833,1
35.0,1,1,53.1,1
54.0,1,0,51.8625,0
4.0,3,1,16.7,1
58.0,1,1,26.55,0
34.0,2,0,13.0,0
28.0,1,0,35.5,0
19.0,1,0,263.0,3
49.0,1,1,76.7292,1
65.0,1,0,61.9792,0


### 3.1 Partitionnement des données

In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33, random_state=10)

In [50]:
print(X.shape)

(183, 5)

In [51]:
print(X_train.shape)

(122, 5)

In [52]:
print(X_test.shape)

(61, 5)

In [53]:
display(X_train)

Age,Pclass,Sex,Fare,SibSp
18.0,1,1,79.65,0
24.0,1,1,49.5042,0
57.0,2,1,10.5,0
42.0,3,0,7.65,0
34.0,2,1,10.5,0
36.0,1,1,71.0,0
38.0,1,0,90.0,1
33.0,1,0,5.0,0
42.0,1,0,26.2875,0
48.0,1,0,52.0,1


### 3.2 Connexion workspace Azure ML service

In [55]:
subscription_id = "70b8f39e-8863-49f7-b6ba-34a80799550c" #you sh70b8f39e-8863-49f7-b6ba-34a80799550could be owner or contributor
resource_group = "azuremlserviceresourcegroup" #you should be owner or contributor
workspace_name = "azuremlservice" #your workspace name
workspace_region = "West Europe" #your region

In [56]:
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)

In [57]:
from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

ws.write_config(path="/databricks/", file_name="config.json")

In [58]:
ws = Workspace.from_config()

experiment_name = 'Titanic_Databricks'
project_folder = './titanic'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

Out[ 40 ]: 
 
Experiment Name Titanic_Databricks 
Location westeurope 
Project Directory ./titanic 
Resource Group azuremlserviceresourcegroup
SDK version 1.0.43 
Workspace Name azuremlservice

In [59]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

Turning diagnostics collection on.

### 3.3 AutoML avec Azure ML service

### Les principes de l'autoML

<img src="https://docs.microsoft.com/en-us/azure/machine-learning/service/media/how-to-automated-ml/automated-machine-learning.png">

> https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml

### Aide en ligne AutoML avec Azure ML service

> https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train

In [64]:
import azureml.dataprep as dprep
import uuid

X_dflow = dprep.read_pandas_dataframe(X_train, temp_folder='/dbfs/tmp'+str(uuid.uuid4()))
y_dflow = dprep.read_pandas_dataframe(y_train, temp_folder='/dbfs/tmp'+str(uuid.uuid4()))

### Paramétrage AutoML

In [66]:
automl_config = AutoMLConfig(task = 'classification', # Type de l'automl
                             debug_log = 'automl_errors.log',
                             primary_metric = 'AUC_weighted', # Métrique de choix des modèles
                             iteration_timeout_minutes = 10, # Timeout
                             iterations = 50, # Nombre itérations
                             n_cross_validations = 5, # Validation croisée
                             max_concurrent_iterations = 8, # A modifier en fonction du nombre de nodes du cluster Spark
                             verbosity = logging.INFO,
                             spark_context=sc, #databricks/spark 
                             X = X_dflow, 
                             y = y_dflow,
                             path = project_folder)

### Exécution autoML Azure ML service

In [68]:
local_run = experiment.submit(automl_config, show_output = True)

Running on ADB cluster experiment Titanic_Databricks.
Parent Run ID: AutoML_f838872c-bdaa-4f84-9a22-ae6d7450fe76
****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION PIPELINE DURATION METRIC BEST
 1 StandardScalerWrapper SGD 0:00:08 0.7824 0.7824
 8 StandardScalerWrapper SGD 0:00:05 0.7964 0.7964
 7 MinMaxScaler RandomForest 0:00:24 0.7767 0.7964
 6 StandardScalerWrapper SGD 0:00:24 0.7905 0.7964
 4 StandardScalerWrapper ExtremeRandomTrees 0:00:24 0.7810 0.7964
 9 MinMaxScaler SGD 0:00:07 0.8033 0.8033
 11 MaxAbsScaler LightGBM 0:00:06 0.8004 0.8033
 10 MinMaxScaler RandomForest 0:00:06 0.8056 0.8056
 12 MinMaxScaler LightGBM 0:00:10 0.7795 0.8056
 16 RobustScaler LogisticRegression 0:00:06 0.7949 0.8056
 15 StandardScalerWrapper BernoulliNaiveBayes 0:00:12 0.8046 0.8056
 14 MinMaxScaler LightGBM 0:00:13 0.7841 0.8056
 13 MinMaxScaler ExtremeRandomTrees 0:00:12 0.8009 0.8056
 18 RobustScaler LogisticRegression 0:00:06 0.7887 0.8056
 17 MaxAbsScaler LinearSVM 0:00:06 0.7366 0.8056
 19 MaxAbsScaler LogisticRegression 0:00:13 0.7806 0.8056
 5 0:01:23 nan 0.8056
 3 StandardScalerWrapper SGD 0:01:31 0.7964 0.8056
 21 RobustScaler KNN 0:00:05 0.6679 0.8056
 20 StandardScalerWrapper RandomForest 0:00:07 0.7932 0.8056
 26 TruncatedSVDWrapper LogisticRegression 0:00:06 0.6465 0.8056
 22 RobustScaler RandomForest 0:00:14 0.7931 0.8056
 2 0:01:34 nan 0.8056
 0 0:01:39 nan 0.8056
 30 MinMaxScaler SGD 0:00:07 0.8020 0.8056
 27 RobustScaler RandomForest 0:00:07 0.7937 0.8056
 25 StandardScalerWrapper KNN 0:00:13 0.6026 0.8056
 24 MinMaxScaler RandomForest 0:00:08 0.7639 0.8056
 23 StandardScalerWrapper LightGBM 0:00:23 0.7850 0.8056
 34 MaxAbsScaler LogisticRegression 0:00:10 0.7709 0.8056
 33 StandardScalerWrapper ExtremeRandomTrees 0:00:08 0.7860 0.8056
 32 StandardScalerWrapper LogisticRegression 0:00:05 0.7964 0.8056
 31 MinMaxScaler LogisticRegression 0:00:12 0.7761 0.8056
 29 SparseNormalizer RandomForest 0:00:19 0.7789 0.8056
 28 MaxAbsScaler LogisticRegression 0:00:15 0.7761 0.8056
 37 RobustScaler LogisticRegression 0:00:11 0.7822 0.8056
 36 StandardScalerWrapper RandomForest 0:00:08 0.7979 0.8056
 35 StandardScalerWrapper RandomForest 0:00:11 0.8102 0.8102
 42 StandardScalerWrapper LogisticRegression 0:00:12 0.7948 0.8102
 41 StandardScalerWrapper GradientBoosting 0:00:13 0.8091 0.8102
 40 RobustScaler LogisticRegression 0:00:05 0.7761 0.8102
 39 RobustScaler LogisticRegression 0:00:09 0.7806 0.8102
 38 StandardScalerWrapper RandomForest 0:00:11 0.8142 0.8142
 47 StandardScalerWrapper LogisticRegression 0:00:05 0.7806 0.8142
 46 StandardScalerWrapper LogisticRegression 0:00:10 0.7887 0.8142
 45 MaxAbsScaler LogisticRegression 0:00:13 0.7793 0.8142
 44 StandardScalerWrapper LogisticRegression 0:00:12 0.7761 0.8142
 43 MinMaxScaler RandomForest 0:00:09 0.7811 0.8142
 49 StackEnsemble 0:00:24 0.8087 0.8142
 48 VotingEnsemble 0:00:22 0.8256 0.8256

### Documentation AutoML :
> https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml

### Liste des modèles :
> https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train

### Visualisation des itérations depuis le portail Azure :

In [71]:
displayHTML("<a href={} target='_blank'> Portail AutoML pour visualiser les résultats : {}</a>".format(local_run.get_portal_url(), local_run.id))

Portail AutoML pour visualiser les résultats : AutoML_f838872c-bdaa-4f84-9a22-ae6d7450fe76

### 3.4 Liste des résultats autoML

In [73]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

Out[ 49 ]: 
 0 1 2 3 ... 46 47 48 49
AUC_macro nan 0.78 nan 0.80 ... 0.79 0.78 0.83 0.81
AUC_micro nan 0.78 nan 0.80 ... 0.79 0.78 0.83 0.81
AUC_weighted nan 0.78 nan 0.80 ... 0.79 0.78 0.83 0.81
accuracy nan 0.73 nan 0.75 ... 0.75 0.71 0.72 0.69
average_precision_score_macro nan 0.87 nan 0.87 ... 0.87 0.87 0.92 0.91
average_precision_score_micro nan 0.87 nan 0.87 ... 0.87 0.87 0.92 0.91
average_precision_score_weighted nan 0.87 nan 0.87 ... 0.87 0.87 0.92 0.91
balanced_accuracy nan 0.71 nan 0.77 ... 0.68 0.69 0.74 0.52
f1_score_macro nan 0.69 nan 0.73 ... 0.68 0.67 0.70 0.44
f1_score_micro nan 0.73 nan 0.75 ... 0.75 0.71 0.72 0.69
f1_score_weighted nan 0.73 nan 0.75 ... 0.74 0.71 0.73 0.58
log_loss nan 0.54 nan 0.64 ... 0.53 0.56 0.52 0.60
norm_macro_recall nan 0.41 nan 0.54 ... 0.37 0.37 0.48 0.04
precision_score_macro nan 0.69 nan 0.74 ... 0.71 0.67 0.71 0.42
precision_score_micro nan 0.73 nan 0.75 ... 0.75 0.71 0.72 0.69
precision_score_weighted nan 0.75 nan 0.80 ... 0.75 0.73 0.77 0.53
recall_score_macro nan 0.71 nan 0.77 ... 0.68 0.69 0.74 0.52
recall_score_micro nan 0.73 nan 0.75 ... 0.75 0.71 0.72 0.69
recall_score_weighted nan 0.73 nan 0.75 ... 0.75 0.71 0.72 0.69
weighted_accuracy nan 0.75 nan 0.73 ... 0.79 0.73 0.71 0.81

[20 rows x 50 columns]

### 3.5 Choix du meilleur modèle selon l'AUC

In [75]:
best_run, fitted_model = local_run.get_output()

print("Meilleur modèle selon l'AUC :")
print("")
print(best_run)
print("")
print(fitted_model)

Meilleur modèle selon l'AUC :

Run(Experiment: Titanic_Databricks,
Id: AutoML_f838872c-bdaa-4f84-9a22-ae6d7450fe76_48,
Type: None,
Status: Completed)

Pipeline(memory=None,
 steps=[('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
 estimators=[('38', Pipeline(memory=None,
 steps=[('StandardScalerWrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x7f33902efb70>), ('RandomForestC...6666666666, 0.40000000000000002, 0.066666666666666666, 0.13333333333333333, 0.066666666666666666]))])

In [76]:
fitted_model.get_params

Out[ 51 ]: 
<bound method Pipeline.get_params of Pipeline(memory=None,
 steps=[('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
 estimators=[('38', Pipeline(memory=None,
 steps=[('StandardScalerWrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x7f33902efb70>), ('RandomForestC...6666666666, 0.40000000000000002, 0.066666666666666666, 0.13333333333333333, 0.066666666666666666]))])>

In [77]:
fitted_model.steps

Out[ 52 ]: 
[('prefittedsoftvotingclassifier',
 PreFittedSoftVotingClassifier(classification_labels=None,
 estimators=[('38', Pipeline(memory=None,
 steps=[('StandardScalerWrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x7f33902efb70>), ('RandomForestClassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
 max_depth=None...tors=100, n_jobs=1,
 oob_score=False, random_state=None, verbose=0, warm_start=False))]))],
 flatten_transform=None,
 weights=[0.066666666666666666, 0.066666666666666666, 0.066666666666666666, 0.066666666666666666, 0.066666666666666666, 0.40000000000000002, 0.066666666666666666, 0.13333333333333333, 0.066666666666666666]))]

### 3.6 Changement métrique de qualité (On utilise maintenant le F1)
##### F1=2*(Precision X Recall)/(Precision+Recall)
> https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9

In [79]:
lookup_metric = "f1_score_weighted"

best_run, fitted_model = local_run.get_output(metric = lookup_metric)

print("Meilleur modèle selon F1:")
print("")
print(best_run)
print("")
print(fitted_model)

Meilleur modèle selon F1:

Run(Experiment: Titanic_Databricks,
Id: AutoML_f838872c-bdaa-4f84-9a22-ae6d7450fe76_3,
Type: None,
Status: Completed)

Pipeline(memory=None,
 steps=[('StandardScalerWrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x7f33908f4320>), ('SGDClassifierWrapper', SGDClassifierWrapper(alpha=0.0001, class_weight='balanced', eta0=0.0001,
 fit_intercept=False, l1_ratio=0.3469387755102041,
 learning_rate='constant', loss='modified_huber', max_iter=1000,
 n_jobs=1, penalty='l2', power_t=0.2222222222222222,
 random_state=None, tol=0.01))])

### 3.7 Choix d'une itération particulière

In [81]:
iteration = 25

my_run, my_model = local_run.get_output(iteration = iteration)

print("Modèle "+ str(iteration) +"ème itération :")
print("")
print(my_run)
print("")
print(my_model)

Modèle 25ème itération :

Run(Experiment: Titanic_Databricks,
Id: AutoML_f838872c-bdaa-4f84-9a22-ae6d7450fe76_25,
Type: None,
Status: Completed)

Pipeline(memory=None,
 steps=[('StandardScalerWrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x7f33904708d0>), ('KNeighborsClassifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
 metric_params=None, n_jobs=1, n_neighbors=7, p=2,
 weights='distance'))])

### 4. On applique le modèle sur la base de Test

### 4.1 On utilise le meilleur modèle selon l'AUC

In [84]:
best_run, fitted_model = local_run.get_output()

print("Meilleur modèle selon l'AUC :")
print("")
print(best_run)
print("")
print(fitted_model)

Meilleur modèle selon l'AUC :

Run(Experiment: Titanic_Databricks,
Id: AutoML_f838872c-bdaa-4f84-9a22-ae6d7450fe76_48,
Type: None,
Status: Completed)

Pipeline(memory=None,
 steps=[('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
 estimators=[('38', Pipeline(memory=None,
 steps=[('StandardScalerWrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x7f33807b1e10>), ('RandomForestC...6666666666, 0.40000000000000002, 0.066666666666666666, 0.13333333333333333, 0.066666666666666666]))])

### 4.2 On applique le modèle

In [86]:
y_test_pred = fitted_model.predict(X_test.values)

In [87]:
Decision=pd.DataFrame(y_test_pred)

In [88]:
# Obtention des scores
proba=fitted_model.predict_proba(X_test.values)

In [89]:
Decision.columns = ['Decision']

In [90]:
Proba=pd.DataFrame(proba)

In [91]:
Proba.columns = ['Proba_0', 'Proba_1']

In [92]:
Proba.head(20)

Out[ 64 ]: 
 Proba_0 Proba_1
0 0.19 0.81 
1 0.18 0.82 
2 0.60 0.40 
3 0.18 0.82 
4 0.20 0.80 
5 0.21 0.79 
6 0.55 0.45 
7 0.58 0.42 
8 0.57 0.43 
9 0.50 0.50 
10 0.54 0.46 
11 0.19 0.81 
12 0.58 0.42 
13 0.51 0.49 
14 0.18 0.82 
15 0.53 0.47 
16 0.55 0.45 
17 0.51 0.49 
18 0.21 0.79 
19 0.18 0.82

In [93]:
Scoring = pd.concat([Decision, Proba], axis=1)

### 4.3 Résultats du scoring

In [95]:
display(Scoring)

Decision,Proba_0,Proba_1
1,0.18871186090210582,0.8112881390978941
1,0.17946290428550843,0.8205370957144915
0,0.5964344270973152,0.40356557290268485
1,0.18076005271810333,0.8192399472818968
1,0.19824825162772544,0.8017517483722747
1,0.21448645657506876,0.7855135434249312
0,0.5462250903215135,0.45377490967848655
0,0.5820155897835759,0.41798441021642413
0,0.5735491524552567,0.42645084754474333
0,0.501769795199915,0.498230204800085


In [96]:
display(Scoring)

Decision,Proba_0,Proba_1
1,0.18871186090210582,0.8112881390978941
1,0.17946290428550843,0.8205370957144915
0,0.5964344270973152,0.40356557290268485
1,0.18076005271810333,0.8192399472818968
1,0.19824825162772544,0.8017517483722747
1,0.21448645657506876,0.7855135434249312
0,0.5462250903215135,0.45377490967848655
0,0.5820155897835759,0.41798441021642413
0,0.5735491524552567,0.42645084754474333
0,0.501769795199915,0.498230204800085


In [97]:
display(Scoring)

Decision,Proba_0,Proba_1
1,0.18871186090210582,0.8112881390978941
1,0.17946290428550843,0.8205370957144915
0,0.5964344270973152,0.40356557290268485
1,0.18076005271810333,0.8192399472818968
1,0.19824825162772544,0.8017517483722747
1,0.21448645657506876,0.7855135434249312
0,0.5462250903215135,0.45377490967848655
0,0.5820155897835759,0.41798441021642413
0,0.5735491524552567,0.42645084754474333
0,0.501769795199915,0.498230204800085


In [98]:
display(Scoring)

Decision,Proba_0,Proba_1
1,0.18871186090210582,0.8112881390978941
1,0.17946290428550843,0.8205370957144915
0,0.5964344270973152,0.40356557290268485
1,0.18076005271810333,0.8192399472818968
1,0.19824825162772544,0.8017517483722747
1,0.21448645657506876,0.7855135434249312
0,0.5462250903215135,0.45377490967848655
0,0.5820155897835759,0.41798441021642413
0,0.5735491524552567,0.42645084754474333
0,0.501769795199915,0.498230204800085


### 4.4 Exportation des scores dans un fichier CSV

In [100]:
# Export CSV
Scoring.to_csv(r'Titanic_scoring.csv')

In [101]:
%sh
ls *.csv -l

-rw-r--r-- 1 root root 2667 Jun 17 13:54 Titanic_scoring.csv

> Fin